<a href="https://colab.research.google.com/github/pbluc3m/collab-R/blob/main/procesar_tfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Parte 0: cojo los ficheros de Olivia y Teresa y los uno

import pandas as pd

input_teresa = 'JASPER_TFE_EPS_23-24.xlsx'
input_olivia = 'JASPER_TFF_COLME_23-24.xls'

df_teresa = pd.read_excel(input_teresa)
df_olivia = pd.read_excel(input_olivia)

columnas = ["NIA", "ALUMNO", "COD.CENTRO", "CENTRO", "COD.EST", "ESTUDIOS", "COD.PLAN", "PLAN", "COD.ASIG", "ASIGNATURA", "COD.OFERTA", "TITULO", "IDIOMA", "NOTA", "CALIF", "SUPERADA", "TUTOR", "DEPARTAMENTO_TUTOR", "TUTOR_EXTERNO"]
df_teresa_c = df_teresa.loc[:, df_teresa.columns.intersection(columnas)]
df_olivia_c = df_olivia.loc[:, df_olivia.columns.intersection(columnas)]

df = pd.concat([df_teresa_c, df_olivia_c], ignore_index=True)

# Vamos a arreglar dos casos identificados

df.loc[df['TUTOR'] == "GONZALEZ CABAÑAS, JOSE", 'DEPARTAMENTO_TUTOR'] = "Departamento de Ingeniería Telemática"
df.loc[df['TUTOR'] == "GOMEZ MARTIN, ADRIAN", 'DEPARTAMENTO_TUTOR'] = "Departamento de Bioingeniería"



In [41]:
# PARTE 1: BASIC STATS

total_rows = df.shape[0]
print(f"Total TFEs: {total_rows}")

# Vamos a quedarnos con los superados
df = df[df["SUPERADA"].str.strip().str.upper() == "S"]
filtered_rows = df.shape[0]
print(f"Total TFEs superados: {filtered_rows}")

# Vamos a quitar los de ADE
df = df[df["ASIGNATURA"] != "Trabajo Fin de Grado (ADE)"]

filtered_rows = df.shape[0]
print(f"Total TFEs sin ADE: {filtered_rows}")


# Por centro
centro = df.groupby("CENTRO").size()
print(centro)

# Por departamento
departamento = df.groupby("DEPARTAMENTO_TUTOR").size()
print(departamento)


estudio_counts = df.groupby("ESTUDIOS").size()

# Display the counts
print(estudio_counts)

Total TFEs: 1305
Total TFEs superados: 1305
Total TFEs sin ADE: 1305
CENTRO
Escuela Politécnica Superior (Colmenarejo)             79
Escuela Politécnica Superior (Leganés)                952
Escuela Politécnica Superior - Centro de Postgrado    274
dtype: int64
DEPARTAMENTO_TUTOR
Departamento de Bioingeniería                                               41
Departamento de Ciencia e Ingeniería de Materiales e Ingenieria Química     24
Departamento de Economía de la Empresa                                       1
Departamento de Estadística                                                 17
Departamento de Física                                                      10
Departamento de Informática                                                236
Departamento de Ingeniería Aeroespacial                                    110
Departamento de Ingeniería Eléctrica                                        81
Departamento de Ingeniería Mecánica                                        141
Departa

In [45]:
df = df[df["CENTRO"] == "Escuela Politécnica Superior (Leganés)"]

# Por departamento
departamento = df.groupby("DEPARTAMENTO_TUTOR").size()
print(departamento)

total_rows = df.shape[0]
print(f"Total TFEs: {total_rows}")

estudio_counts = df.groupby("ESTUDIOS").size()

# Display the counts
print(estudio_counts)

DEPARTAMENTO_TUTOR
Departamento de Bioingeniería                                               41
Departamento de Ciencia e Ingeniería de Materiales e Ingenieria Química     20
Departamento de Economía de la Empresa                                       1
Departamento de Estadística                                                 17
Departamento de Física                                                      10
Departamento de Informática                                                153
Departamento de Ingeniería Aeroespacial                                     69
Departamento de Ingeniería Eléctrica                                        61
Departamento de Ingeniería Mecánica                                         99
Departamento de Ingeniería Telemática                                       83
Departamento de Ingeniería Térmica y de Fluidos                             92
Departamento de Ingeniería de Sistemas y Automática                         57
Departamento de Matemáticas      

In [ ]:

# Vamos a quedarnos solo con unas columnas

columns_to_keep = ["NIA", "ALUMNO", "CENTRO", "ESTUDIOS", "OFERTA", "TITULO", "TUTOR", "DEPARTAMENTO_TUTOR", "TUTOR_EXTERNO"]
df_columns = df_superada.loc[:, df_superada.columns.intersection(columns_to_keep)]

output_file = 'JASPER_TOTAL-23-24-clean.xlsx'
df_columns.to_excel(output_file, index=False, engine="openpyxl")


In [21]:
# PARTE 2: SOLO "DEPARTAMENTO DE" Y GENERAMOS UN ARCHIVO POR DEPARTAMENTO
import unicodedata

def remove_accents(text):
    if isinstance(text, str):
        return "".join(c for c in unicodedata.normalize("NFD", text) if unicodedata.category(c) != "Mn")
    return text

input_file_clean = 'JASPER_TOTAL-23-24-clean.xlsx'
df = pd.read_excel(input_file_clean)

df["DEPARTAMENTO_TUTOR"] = df["DEPARTAMENTO_TUTOR"].apply(remove_accents)

# Estadisticas
total_rows = df.shape[0]
print(f"Total number of rows: {total_rows}")

filtered_df = df[df["DEPARTAMENTO_TUTOR"].str.startswith("Departamento de", na=False)]
total_rows = filtered_df.shape[0]
print(f"Total number of rows where Departamento de: {total_rows}")


output_folder = "TFE-Departamentos"
os.makedirs(output_folder, exist_ok=True)

for department, group_df in filtered_df.groupby("DEPARTAMENTO_TUTOR"):
    safe_filename = f"{department}.xlsx".replace("/", "_").replace(" ", "_")
    file_path = os.path.join(output_folder, safe_filename)
    group_df.to_excel(filename, index=False, engine="openpyxl")
    print(f"Saved {filename}")


Total number of rows: 1339
Total number of rows where Departamento de: 1332
Saved Departamento_de_Bioingenieria.xlsx
Saved Departamento_de_Ciencia_e_Ingenieria_de_Materiales_e_Ingenieria_Quimica.xlsx
Saved Departamento_de_Economia_de_la_Empresa.xlsx
Saved Departamento_de_Estadistica.xlsx
Saved Departamento_de_Fisica.xlsx
Saved Departamento_de_Informatica.xlsx
Saved Departamento_de_Ingenieria_Aeroespacial.xlsx
Saved Departamento_de_Ingenieria_Electrica.xlsx
Saved Departamento_de_Ingenieria_Mecanica.xlsx
Saved Departamento_de_Ingenieria_Telematica.xlsx
Saved Departamento_de_Ingenieria_Termica_y_de_Fluidos.xlsx
Saved Departamento_de_Ingenieria_de_Sistemas_y_Automatica.xlsx
Saved Departamento_de_Matematicas.xlsx
Saved Departamento_de_Mecanica_de_Medios_Continuos_y_Teoria_de_Estructuras.xlsx
Saved Departamento_de_Tecnologia_Electronica.xlsx
Saved Departamento_de_Teoria_de_la_Senal_y_Comunicaciones.xlsx


In [22]:
estudio_counts = filtered_df.groupby("ESTUDIOS").size()

# Display the counts
print(estudio_counts)

ESTUDIOS
Alumnos de Programas de Intercambio                                                      1
Doble Grado en Ingeniería Física e Ingeniería en Tecnologías Industriales                1
Doble Grado en Ingeniería Informática y Administración de Empresas                      62
Grado en Ciencia e Ingeniería de Datos                                                  44
Grado en Ciencias                                                                        1
Grado en Ingeniería Aeroespacial                                                        89
Grado en Ingeniería Biomédica                                                           51
Grado en Ingeniería Electrónica Industrial y Automática                                 94
Grado en Ingeniería Eléctrica                                                           32
Grado en Ingeniería Física                                                              22
Grado en Ingeniería Informática                                                  